# Imports

In [13]:
#Importing libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import joblib
sources_path = './../Sources'
if sources_path not in sys.path:
    sys.path.append(sources_path)
from messaging.telegrambot import Bot
import telegram
from Evaluations.metrics import MSE, RMSE, MAE
from Environment import io
devooght_package = os.path.join(sources_path, 'Devooght')
if devooght_package not in sys.path:
    sys.path.append(devooght_package)
import Devooght.preprocess as preprocess
reload(preprocess)
from Recommender import recsys as recsys
reload(recsys)
#sys.path.append('../Sources') # Adding my own libs source
#from StackedAutoEncoderRS.SAERS import SAE

<module 'Recommender.recsys' from './../Sources/Recommender/recsys.pyc'>

In [ ]:
reload(io)

# Defining Constant Variables

In [9]:
%matplotlib inline
results_path = './Results/'
verbose = True

# Loading Dataset

In [10]:
#Loading dataset
dataset = '100k/' #100k dataset path
#dataset = '10M/' #10M dataset path
#dataset = '20M/' #20M dataset path

dataset_path = '../../Datasets/movieLens/' + dataset # Full dataset path

filenames = {'movie': 'movie.csv', 'rating': 'rating.csv'}

[Loading Time: 0.0948848724365 s] rating.csv Shape:  (100000, 4)


In [16]:
df_ratings = preprocess.load_data(
                filename = dataset_path+filenames['rating'], 
                columns = ['userId', 'itemId', 'rating', 'timestamp'], 
                separator = ',',
                sort_time = False,
                verbose = verbose)    

[*] Loading data from ../../Datasets/movieLens/100k/rating.csv...
[ ] Sorting timestamp column
[+] Data Loaded
   userId  itemId  rating           timestamp
0     196     242       3 1997-12-04 15:55:49
1     186     302       3 1998-04-04 19:22:22
2      22     377       1 1997-11-07 07:18:36
3     244      51       2 1997-11-27 05:02:03
4     166     346       1 1998-02-02 05:33:16


In [17]:
df_ratings.head()

,userId,itemId,rating,timestamp
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


# Creating Recommender Class

# Creating Recommender Instance

In [18]:
%%time
reload(recsys)
rs = recsys.recommender(algorithm = 'fism')
print rs.get_name()
#rs.fit(df_ratings = df_ratings, 
#       k = 40,
#       n_epochs = 10, 
#       n_factors = 40, 
#       columns = ['userId', 'itemId', 'rating', 'timestamp'],
#       verbose = True)


fism
CPU times: user 6.7 ms, sys: 495 µs, total: 7.19 ms
Wall time: 24.7 ms


In [19]:
rs.fit(df_ratings = df_ratings,       
       verbose = verbose)

[*] Creating directories in ./Sequence_based_recommendation_files/fism
[ ] Creating data directory
[ ] Creating models directory
[ ] Creating results directory
[+] Directories created.
[*] Removing inactive users and rare items...
[+] Rare users and items removed.
[*] Splitting data into training, validation and test sets
[ ] Saving training, validation and test sets in the triplets format
[+] Split data saved on ./Sequence_based_recommendation_files/fism/data folder.
[*] Making sequence format
[ ] Saving training set in the sequences format...
[ ] Saving validation set in the sequences format...
[ ] Saving test set in the sequences format...
[ ] Saving the extended training set in the sequences format...
[+] Sequence format saved on ./Sequence_based_recommendation_files/fism/data folder
[*] Saving stats to ./Sequence_based_recommendation_files/fism/data/stats file
[+] Stats saved.
RMSE SGD delta:  11.0799920572
RMSE SGD delta:  -0.0852666676217
RMSE SGD delta:  -9.41782020602
RMSE SGD

10 0.00012662555557 0.856633901596 -5.84759805995 0.00536963186423 0.0 0.1 27 0.0359564138785 0.295454545455


0

In [21]:
rs.get_metrics()

({'blockbuster_share': 0.1,
  'item_coverage': 27,
  'ndcg': 0.035956413878503318,
  'recall': 0.005369631864229946,
  'sps': 0.0,
  'user_coverage': 0.29545454545454547},
 0.9779658317565918,
 './Sequence_based_recommendation_files/fism/models/fism_RMSE_ne0.0_lr0.01_an1.0_k20_reg0.0_ini1.npz')

# Getting Predictions for a Single User

In [ ]:
#rs.get_top_n(10)
#rs.get_model()
print (rs.predict(userId = 196, itemId = 242))

## Getting Predictions of Unknown Ratings

In [ ]:
#rs.calculate_unknown_prediction()

In [ ]:
%%time
#df_unknown_predictions = rs.get_unknown_predictions(calculate_predictions = False)

## Getting Predictions of Known Ratings

In [ ]:
%%time
df_known_predictions = rs.get_known_predictions(calculate_predictions = False)

In [ ]:
df_known_predictions.head()

## Building users topN Recommendation

In [ ]:
%%time
known_sequences = rs.get_top_n(n=10, source = 'known', calculate_sequence = True)

In [ ]:
print (len(known_sequences))
#known_sequences

## Evaluating Algorithm

In [ ]:
print (MSE(df_known_predictions['rating'], df_known_predictions['prediction']))
print (RMSE(df_known_predictions['rating'], df_known_predictions['prediction']))
print (MAE(df_known_predictions['rating'], df_known_predictions['prediction']))

df_known_predictions.head()

## Evaluating Number of Neighbours on KNN-Based-Algorithms

In [ ]:
%%time
knn_based_algorithms = ['knnbasic', 'kmeans']
k_array = np.arange(1, 301, 5)

for algorithm in knn_based_algorithms:    
    time_zero = time.time()
    rs = recsys.recommender(algorithm = algorithm)
    print ("----------------------------------------------------")
    print ("Calculating evaluations for {}".format(rs.get_name()))
    
    #knn_predictions = dict()
    k_evaluations = dict()
    
    for k in k_array:            
        # Fitting Algorithm
        rs.fit(df_ratings = df_ratings, k = k, verbose = False)
        
        # Getting Known Predictions
        df_known_predictions = rs.get_known_predictions(calculate_predictions = True)
        
        # Evaluating Known Predictions
        mae = MAE(df_known_predictions['rating'], df_known_predictions['prediction'])
        rmse = RMSE(df_known_predictions['rating'], df_known_predictions['prediction'])
        mse = MSE(df_known_predictions['rating'], df_known_predictions['prediction'])
        
        k_evaluations[k] = {'mae': mae, 'rmse': rmse, 'mse': mse}
        
    # Saving evaluations
    saveVariable(results_path = results_path, dataset = dataset, variable = k_evaluations, filename = '{}_evaluations_dic'.format(algorithm), verbose = True)
    print ("Elapsed time for {}: {}".format(rs.get_name(), time.time()-time_zero))

In [ ]:
# Loading algorithm evaluations variable
for algorithm in knn_based_algorithms:
    #algorithm = 'kmeans'#'knnbasic'
    k_evaluations = loadVariable(results_path=results_path, dataset = dataset, filename = '{}_evaluations_dic'.format(algorithm), verbose = verbose)

    # Converting Dictionary to array
    k_array = np.array([])
    rmse_array = np.array([])
    mae_array = np.array([])
    mse_array = np.array([])

    for key in sorted(k_evaluations.keys()):
        k_array = np.append(k_array, int(key))
        rmse_array = np.append(rmse_array, k_evaluations[key]['rmse'])
        mse_array = np.append(mse_array, k_evaluations[key]['mse'])
        mae_array = np.append(mae_array, k_evaluations[key]['mae'])

    # Plotting Evaluations
    fig, ax = plt.subplots(figsize=(15, 15))
    ax.plot(k_array, rmse_array, label = 'rmse')
    ax.plot(k_array, mse_array, label = 'mse')
    ax.plot(k_array, mae_array, label = 'mae')
    #ax.axis([0, k_array[-1], 0, itemsQuantity[-1]])
    ax.grid(True)
    ax.legend(fontsize = 14)
    ax.set_xticks([x for x in xrange (int(k_array[0]), int(k_array[-1])+1, 15)])
    #ax.set_yticks([x for x in xrange (0, itemsQuantity[-1], 100)])

    ax.set_xlabel('Number of Neighbours', fontsize = 20)
    ax.set_ylabel('Evaluation', fontsize = 20)
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.set_title('Evaluations x Neighbours - {}'.format(algorithm.title()), fontsize = 20)

    io.saveFigure(results_path, dataset, '{}_evaluations_per_neighbours.png'.format(algorithm), fig, verbose = True)

## Evaluating Number of Factors on Matrix Factorization-based Algorithms

In [ ]:
%%time
matrix_factorization_based_algorithms = ['svd']
n_factors_array = np.arange(70, 311, 20)


for algorithm in matrix_factorization_based_algorithms:    
    time_zero = time.time()
    rs = recsys.recommender(algorithm = algorithm)
    print ("----------------------------------------------------")
    print ("Calculating evaluations for {}".format(rs.get_name()))
    
    matrix_factorization_evaluations = dict()
    
    for n_factors in n_factors_array:            
        # Fitting Algorithm
        rs.fit(df_ratings = df_ratings, n_factors = n_factors, verbose = False)
        
        # Getting Known Predictions
        df_known_predictions = rs.get_known_predictions(calculate_predictions = True)
        
        # Evaluating Known Predictions
        mae = MAE(df_known_predictions['rating'], df_known_predictions['prediction'])
        rmse = RMSE(df_known_predictions['rating'], df_known_predictions['prediction'])
        mse = MSE(df_known_predictions['rating'], df_known_predictions['prediction'])
        
        matrix_factorization_evaluations[n_factors] = {'mae': mae, 'rmse': rmse, 'mse': mse}
        
    # Saving evaluations
    saveVariable(results_path = results_path, dataset = dataset, variable = matrix_factorization_evaluations, filename = '{}_evaluations_dic'.format(algorithm), verbose = True)
    print ("Elapsed time for {}: {}".format(rs.get_name(), time.time()-time_zero))

In [ ]:
# Loading algorithm evaluations variable
for algorithm in matrix_factorization_based_algorithms:
    #algorithm = 'kmeans'#'knnbasic'
    n_factors_evaluations = loadVariable(results_path=results_path, dataset = dataset, filename = '{}_evaluations_dic'.format(algorithm), verbose = verbose)

    # Converting Dictionary to array
    n_factors_array = np.array([])
    rmse_array = np.array([])
    mae_array = np.array([])
    mse_array = np.array([])

    for key in sorted(n_factors_evaluations.keys()):
        n_factors_array = np.append(n_factors_array, int(key))
        rmse_array = np.append(rmse_array, n_factors_evaluations[key]['rmse'])
        mse_array = np.append(mse_array, n_factors_evaluations[key]['mse'])
        mae_array = np.append(mae_array, n_factors_evaluations[key]['mae'])

    # Plotting Evaluations
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.plot(n_factors_array, rmse_array, label = 'rmse')
    ax.plot(n_factors_array, mse_array, label = 'mse')
    ax.plot(n_factors_array, mae_array, label = 'mae')
    #ax.axis([0, k_array[-1], 0, itemsQuantity[-1]])
    ax.grid(True)
    ax.legend(fontsize = 14)
    ax.set_xticks([x for x in xrange (int(n_factors_array[0]), int(n_factors_array[-1])+1, 20)])
    #ax.set_yticks([x for x in xrange (0, itemsQuantity[-1], 100)])

    ax.set_xlabel('Number of Factors', fontsize = 20)
    ax.set_ylabel('Evaluation', fontsize = 20)
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.set_title('Evaluations x Number of Factors - {}'.format(algorithm.title()), fontsize = 15)

    saveFigure(results_path, dataset, '{}_evaluations_per_factors.png'.format(algorithm), fig, verbose = True)